# Tips to improve training stability

## What's the list of all the tricks and techniques used to improve stability?

#### Note: I forgot the source, these super useful tips were shared in some discord server and at the time I only captured an image of that message.

- Gradient clipping (I like 1.0)

- muP or just better inits (e.g. std 0.02 is too high for 100B+ models, and I can vouch that something like 0.005 works well) https://arxiv.org/abs/2201.11990

- LR + batch size tuning (I try not to exceed 4-8M tokens for global batch size on vanilla setups; it's higher when batches are split across experts)

- Higher precision (I've never seen a model that needs more than mixed-precision bf16+fp16; sometimes AllReduce/ReduceScatter can require fp32)

- Z-Loss from palm

- LayerNorm instead of RMSNorm (RMSNorm appears to lead to larger activations across layers; LayerNorm is more of a thing I'm seeing recently across arch types (dense, ssm, moe). Take from that what you will.)

- Sometimes spikes are legitimate and caused by challenging or bad samples (imagine a sample misclassified or a bad sample mixed into an addition dataset). Most of those spikes are recoverable, but if it kicks you out of a region you like, just hack the dataloader to skip those samples.

- There's a weird property in adam that your optim states start falling into an unstable region sometimes, signaled by a slowly increasing grad norm, and a sudden loss spike that you don't fully recover from. It's recoverable by rewinding to an earlier ckpt and throwing away optimizer states.

- If you're using adam, try to lower **Beta2 to 0.95**. Negligibly slower convergence but much more stable. Reduces chance of states falling into unstable regions. See section 3.4 of https://arxiv.org/pdf/2304.13013.pdf

- **Disabling linear biases** seems to cap the growth of key projection bias weights so that they don't dominate and affect overall though.

## Summary of some discussion on fast.ai discord server

1. We use a combination of Dice and Cross-Entropy (CE) loss, with Dice loss resembling IOU loss. This significantly boosts performance, especially with class imbalances.

2. We oversample foreground patches, which increases training stability, as many patches in datasets can be empty during patch-based training.

3. Each dataset is unique, so we avoid a fixed learning rate schedule and do not use the validation set for realistic cross-validation. Instead, we employ a `ReduceLROnPlateau` scheduler based on an exponential moving average of training loss.

4. Data augmentation is crucial. We developed our own [data augmentation framework](https://github.com/MIC-DKFZ/batchgenerators) capable of handling 3D data efficiently with multiprocessing.

5. We utilize a simple UNet architecture with standard convolutions. While advanced architectures like ResNets and DenseNets exist, they often require more VRAM, limiting patch size or feature maps. We haven't found a network that outperforms the plain UNet given our hardware constraints.

6. Regarding normalization, we prefer Instance Norm over Batch Norm due to unreliable batch statistics with small batch sizes. We observed similar performance with Instance Norm, and it conserves VRAM compared to Group Norm. Normalization is essential for loss convergence, particularly with Dice loss.

7. Our model is optimized for the medical domain and may not transfer well to other computer vision areas. We plan to adapt nnUNet for 2D natural images and conduct experiments on the Cityscapes dataset.